In [14]:
import requests
from bs4 import BeautifulSoup
for year in range(2006,2018):
    url = "https://www.billboard.com/charts/year-end/"+ str(year)+"/hot-100-songs"
    response = requests.get(url)
    results_page = BeautifulSoup(response.content,'lxml')
    # print(results_page.prettify())
    article_tags = results_page.find_all('article', class_= "ye-chart-item")
    for tag in article_tags:
        print ((tag.find('div', class_= "ye-chart-item__artist").get_text().strip(), tag.find('div', class_= "ye-chart-item__title").get_text().strip()))

    


('Daniel Powter', 'Bad Day')
('Sean Paul', 'Temperature')
('Nelly Furtado Featuring Timbaland', 'Promiscuous')
('James Blunt', "You're Beautiful")
('Shakira Featuring Wyclef Jean', "Hips Don't Lie")
('Natasha Bedingfield', 'Unwritten')
('Gnarls Barkley', 'Crazy')
('Chamillionaire Featuring Krayzie Bone', "Ridin'")
('Justin Timberlake', 'SexyBack')
('Beyonce Featuring Slim Thug', 'Check On It')
('Mary J. Blige', 'Be Without You')
('Nelly Featuring Paul Wall, Ali & Gipp', 'Grillz')
('The Fray', 'Over My Head (Cable Car)')
('Cassie', 'Me & U')
('The Pussycat Dolls Featuring Snoop Dogg', 'Buttons')
('Chris Brown', 'Run It!')
('Ne-Yo', 'So Sick')
('Yung Joc', "It's Goin' Down")
('Rihanna', 'SOS')
('Panic! At The Disco', 'I Write Sins Not Tragedies')
('The All-American Rejects', 'Move Along')
('Fergie', 'London Bridge')
('Red Hot Chili Peppers', 'Dani California')
('Lil Jon Featuring E-40 & Sean Paul Of The YoungBloodZ', 'Snap Yo Fingers')
('Dem Franchize Boyz Featuring Lil Peanut & Charlay'

('Ke$ha', 'TiK ToK')
('Lady Antebellum', 'Need You Now')
('Train', 'Hey, Soul Sister')
('Katy Perry Featuring Snoop Dogg', 'California Gurls')
('Usher Featuring will.i.am', 'OMG')
('B.o.B Featuring Hayley Williams', 'Airplanes')
('Eminem Featuring Rihanna', 'Love The Way You Lie')
('Lady Gaga', 'Bad Romance')
('Taio Cruz', 'Dynamite')
('Taio Cruz Featuring Ludacris', 'Break Your Heart')
('B.o.B Featuring Bruno Mars', "Nothin' On You")
('Enrique Iglesias Featuring Pitbull', 'I Like It')
('Young Money Featuring Lloyd', 'BedRock')
('Jason Derulo', 'In My Head')
('Rihanna', 'Rude Boy')
('Lady Gaga Featuring Beyonce', 'Telephone')
('Katy Perry', 'Teenage Dream')
('Bruno Mars', 'Just The Way You Are')
('Mike Posner', 'Cooler Than Me')
('The Black Eyed Peas', 'Imma Be')
('Jay-Z + Alicia Keys', 'Empire State Of Mind')
('Usher Featuring Pitbull', "DJ Got Us Fallin' In Love")
('Travie McCoy Featuring Bruno Mars', 'Billionaire')
('Eminem', 'Not Afraid')
('Iyaz', 'Replay')
('David Guetta Featuring

('Selena Gomez', 'Come & Get It')
('Drake Featuring Majid Jordan', "Hold On, We're Going Home")
('Maroon 5', 'Daylight')
('Pitbull Featuring Christina Aguilera', 'Feel This Moment')
('Lady Gaga', 'Applause')
('Maroon 5', 'One More Night')
('Lil Wayne Featuring Drake & Future', 'Love Me')
('Fall Out Boy', 'My Songs Know What You Did In The Dark (Light Em Up)')
('A$AP Rocky Featuring Drake, 2 Chainz & Kendrick Lamar', 'F**kin Problems')
('Justin Bieber Featuring Nicki Minaj', 'Beauty And A Beat')
('Macklemore & Ryan Lewis Featuring Mary Lambert', 'Same Love')
('Calvin Harris Featuring Florence Welch', 'Sweet Nothing')
('Lana Del Rey & Cedric Gervais', 'Summertime Sadness')
('Phillip Phillips', 'Home')
('Imagine Dragons', "It's Time")
('J. Cole Featuring Miguel', 'Power Trip')
('Alicia Keys Featuring Nicki Minaj', 'Girl On Fire')
('Demi Lovato', 'Heart Attack')
('Maroon 5', 'Love Somebody')
('Mumford & Sons', 'I Will Wait')
('P!nk', 'Try')
('Darius Rucker', 'Wagon Wheel')
('PSY', 'Gangnam

('Ed Sheeran', 'Shape Of You')
('Luis Fonsi & Daddy Yankee Featuring Justin Bieber', 'Despacito')
('Bruno Mars', "That's What I Like")
('Kendrick Lamar', 'Humble.')
('The Chainsmokers & Coldplay', 'Something Just Like This')
('Migos Featuring Lil Uzi Vert', 'Bad And Boujee')
('The Chainsmokers Featuring Halsey', 'Closer')
('Sam Hunt', 'Body Like A Back Road')
('Imagine Dragons', 'Believer')
('Post Malone Featuring Quavo', 'Congratulations')
('James Arthur', "Say You Won't Let Go")
('DJ Khaled Featuring Justin Bieber, Quavo, Chance The Rapper & Lil Wayne', "I'm The One")
('Lil Uzi Vert', 'XO TOUR Llif3')
('Future', 'Mask Off')
('French Montana Featuring Swae Lee', 'Unforgettable')
('Bruno Mars', '24K Magic')
('Zedd & Alessia Cara', 'Stay')
('DJ Khaled Featuring Rihanna & Bryson Tiller', 'Wild Thoughts')
('Rae Sremmurd Featuring Gucci Mane', 'Black Beatles')
('The Weeknd Featuring Daft Punk', 'Starboy')
('Khalid', 'Location')
('Charlie Puth', 'Attention')
('Shawn Mendes', "There's Nothin

In [6]:
import requests
from bs4 import BeautifulSoup
url = "https://www.billboard.com/charts/hot-100"
response = requests.get(url)
results_page = BeautifulSoup(response.content,'lxml')
# print(results_page.prettify())
div_tags = results_page.find_all('div', class_= "chart-list-item")
for div in div_tags:
    print((div.get("data-artist"), div.get("data-title")))
        

('Travis Scott', 'Sicko Mode')
('Juice WRLD', 'Lucid Dreams')
('Marshmello & Bastille', 'Happier')
('Post Malone', 'Better Now')
('Kodak Black Featuring Travis Scott & Offset', 'ZEZE')
('5 Seconds Of Summer', 'Youngblood')
('Lil Baby & Gunna', 'Drip Too Hard')
('Post Malone & Swae Lee', 'Sunflower (Spider-Man: Into The Spider-Verse)')
('Khalid & Normani', 'Love Lies')
('DJ Snake Featuring Selena Gomez, Ozuna & Cardi B', 'Taki Taki')
('Halsey', 'Without Me')
('Bad Bunny Featuring Drake', 'MIA')
('Lady Gaga & Bradley Cooper', 'Shallow')
('Imagine Dragons', 'Natural')
('Sheck Wes', 'Mo Bamba')
('Cardi B, Bad Bunny & J Balvin', 'I Like It')
('Ella Mai', 'Trip')
('Tyga Featuring Offset', 'Taste')
('Benny Blanco, Halsey & Khalid', 'Eastside')
('Panic! At The Disco', 'High Hopes')
('Drake', 'Nonstop')
('Drake', 'In My Feelings')
('Ariana Grande', 'God Is A Woman')
('6ix9ine Featuring Nicki Minaj & Murda Beatz', 'FEFE')
('Future & Juice WRLD', 'Fine China')
('Lil Baby & Drake', 'Yes Indeed')
(

In [17]:
# !pip3 install spotipy


  Using cached https://files.pythonhosted.org/packages/59/46/3c957255c96910a8a0e2d9c25db1de51a8676ebba01d7966bedc6e748822/spotipy-2.4.4.tar.gz
  Running setup.py bdist_wheel for spotipy ... done
  Stored in directory: /Users/shazliekhan/Library/Caches/pip/wheels/76/28/19/a86ca9bb0e32dbd4a4f580870250f5aeef852870578e0427e6
Successfully built spotipy


In [35]:
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id="a31e2c1446fd4e6aa11476d8532fc939",client_secret="b52f884f9d4f464eba58daf0d5180ea4")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
from collections import OrderedDict
def get_spotify_features(track, artist):
    #Search for Spofity song ID 
    songs=sp.search(q='track:'+track+' '+'artist:'+artist+'*' , type='track')
    items = songs['tracks']['items']
    if len(items) ==0:
        return([0]*len(features))
    else:
        track = items[0]
        song_id = str(track["id"])
        #Use ID to get Song features
        track_features=sp.audio_features(song_id)
        if len(track_features[0]) <18:
            return([0]*len(features))
        else:
            features_to_df = np.array(track_features)[0]
            #Order Dictionary
            features_to_df = OrderedDict(features_to_df)
            return (features_to_df)
            #Get Dictionary values
            feature_values = features_to_df.values()
#           return(feature_values)
get_spotify_features("Sicko Mode","Travis Scott")

OrderedDict([('danceability', 0.834),
             ('energy', 0.73),
             ('key', 8),
             ('loudness', -3.714),
             ('mode', 1),
             ('speechiness', 0.222),
             ('acousticness', 0.00513),
             ('instrumentalness', 0),
             ('liveness', 0.124),
             ('valence', 0.446),
             ('tempo', 155.008),
             ('type', 'audio_features'),
             ('id', '2xLMifQCjDGFmkHkpNLD9h'),
             ('uri', 'spotify:track:2xLMifQCjDGFmkHkpNLD9h'),
             ('track_href',
              'https://api.spotify.com/v1/tracks/2xLMifQCjDGFmkHkpNLD9h'),
             ('analysis_url',
              'https://api.spotify.com/v1/audio-analysis/2xLMifQCjDGFmkHkpNLD9h'),
             ('duration_ms', 312820),
             ('time_signature', 4)])